In [26]:
# !uvicorn causal.text_generation_api:app --reload --host 127.0.0.1 --port 8889

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import pandas as pd
from datasets import load_from_disk
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import requests
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from langchain import PromptTemplate
from utils.evaluate import calculate_cosine_similarity
from tqdm import tqdm
import psutil
import subprocess
import signal
import re

/home/llama/Personal_Directories/srb/causalEnv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
! which python

~/Personal_Directories/srb/causalEnv/bin/python


Check if the api is running

In [3]:
def find_uvicorn_processes():
    uvicorn_processes = []
    for process in psutil.process_iter(['pid', 'name', 'cmdline']):
        if 'uvicorn' in process.info['name'].lower() or \
                ('uvicorn' in process.info['cmdline'] if process.info['cmdline'] else False):
            uvicorn_processes.append(process.info)
 
    return uvicorn_processes
 
def kill_process_by_pid(pid):
    try:
        process = psutil.Process(pid)
        process.terminate()  # Alternatively, use process.kill() for a forceful termination
        print(f"Process {pid} has been terminated.")
    except psutil.NoSuchProcess:
        print(f"No process found with PID {pid}.")

uvicorn_processes = find_uvicorn_processes()

if uvicorn_processes:
    print("The follwoing is ruuning: ")
    for process in uvicorn_processes:
        print(f"PID: {process['pid']}, Name: {process['name']}, Command: {' '.join(process['cmdline'])}")
else:
    print("Uvicore unfound")

The follwoing is ruuning: 
PID: 24290, Name: uvicorn, Command: /home/llama/Sandbox_IA/env/bin/python /home/llama/Sandbox_IA/env/bin/uvicorn app.main:app --reload --port 8001


Open the server using uvicorn

In [19]:
directory = "causal"
module = "text_generation_api_mixtral"
model = "Mixtral_8_7B"
host = "127.0.0.1"
port = 8899
quantization = "original"
python_executable = "~/Personal_Directories/srb/causalEnv/bin/python"
test_num = 300

In [5]:
import subprocess
from concurrent.futures import ThreadPoolExecutor
 
# %nohup run_uvicorn_command(directory, your_module, host,port)
uvicorn_command = f"{python_executable} -m uvicorn {directory}.{module}:app --reload --host {host} --port {port}"
subprocess.Popen(uvicorn_command, shell=True)

<Popen: returncode: None args: '~/Personal_Directories/srb/causalEnv/bin/pyt...>

INFO:     Will watch for changes in these directories: ['/home/llama/Personal_Directories/srb/causalllm-main']
INFO:     Uvicorn running on http://127.0.0.1:8899 (Press CTRL+C to quit)
INFO:     Started reloader process [1219466] using StatReload
Loading checkpoint shards:  53%|█████▎    | 10/19 [00:04<00:04,  2.07it/s]

In [12]:
def extract_answer(input_string):
    match = re.search(r'<Answer>(.*?)<\/Answer>', input_string)
    if match:
        answer = match.group(1)
        return answer, True
    else:
        return input_string, False  
# answer,_ = extract_answer(input_string)
# print(answer)

# model_id = "/home/llama/Personal_Directories/srb/causalllm-main/model/Llama-2-13b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(
#     model_id
# )
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     device_map="auto")

In [13]:
# Define template
system_prompt = """
Please strictly answer my question, but format it using custom tags like <Answer>Your answer here<Answer>. If the question cannot be answered, just answer with "I don't know". 
"""
user_message = """

Instruction: Please strictly answer the question, but format it using custom tags like <Answer>Your answer here<Answer>. If the question cannot be answered, just answer with "I don't know". 
Context: Swift has won 12 Grammy Awards (including three for Album of the Year—tying for the most by an artist), an Emmy Award, 40 American Music Awards (the most won by an artist),29 Billboard Music Awards (the most won by a woman), 92 Guinness World Records, 14 MTV Video Music Awards (including three Video of the Year wins—the most by an act), 12 Country Music Association Awards (including the Pinnacle Award), eight Academy of Country Music Awards, and two Brit Awards. As a songwriter, she has been honored by the Nashville Songwriters Association, the Songwriters Hall of Fame, and the National Music Publishers' Association and was the youngest person on Rolling Stone's list of the 100 Greatest Songwriters of All Time in 2015. At the 64th BMI Awards in 2016, Swift was the first woman to be honored with an award named after its recipient. In 2023, Swift became the first act to be named the global best-selling artist of the year by the International Federation of the Phonographic Industry (IFPI) thrice.

From available data, Swift has amassed over 50 million album sales, 150 million single sales, and 114 million units globally, including 78 billion streams. The International Federation of the Phonographic Industry (IFPI) ranked her as the Global Recording Artist of the Year for a record three times (2014, 2019 and 2022). Swift has the most number-one albums in the United Kingdom and Ireland for a female artist this millennium and earned the highest income for an artist on Chinese digital music platforms—RMB 159,000,000. Swift is the most streamed female act on Spotify, and the only artist to have received more than 200 million streams in one day (228 million streams on October 21, 2022).The most entries and the most simultaneous entries for an artist on the Billboard Global 200, with 94 and 31 songs, respectively, are among her feats. Her Reputation Stadium Tour (2018) is the highest-grossing North American tour ever, and she was the world's highest-grossing female touring act of the 2010s. Beginning with Fearless, all of her studio albums opened with over a million global units.
Question: Given the passage below, tell me how many Grammy Awards, Emmy Awards, and Billboard Music Awards Taylor Swift has won and how many simultaneous entries she had on the Billboard Global 200 when she had the most simultaneous entries for an artist.

"""
template = """
<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{user_message} [/INST]

"""
prompt_template = PromptTemplate.from_template(template)
# prompt = prompt_template.format(
#     system_prompt=system_prompt, user_message=user_message
# )

# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
# generated_ids = model.generate(
#     **inputs
# )

# extract_answer(input_string)


Prepare dataset

In [14]:
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 20)
pd.set_option("max_colwidth", 1000)

local_dataset_path = "resource/data/databricks-dolly-15k"
dataset = load_from_disk(local_dataset_path)
df = dataset.to_pandas()
test_df = df[df["category"] == "closed_qa"]
test = test_df.head(test_num)

In [22]:
url = "http://127.0.0.1:8899/generate-text"
system_prompt = """
Please strictly answer my question, but format it using custom tags like <Answer>Your answer here<Answer>. If the question cannot be answered, just answer with "I don't know". """
# template = """
# <s>[INST] <<SYS>>
# {system_prompt}
# <</SYS>>

# Context: {context_message}
# Question: {question_message}

# [/INST]

# """

template = """
Instruction:{system_prompt}
Context: {context_message}
Question: {question_message}
"""
prompt_template = PromptTemplate.from_template(template)

columns = [
    "temperature",
    "model",
    "ground_truth_response",
    "prompt"
]
result_df = pd.DataFrame(columns=columns)

# build the preprocessed csv files
for index, row in test.iterrows():
    question = row["instruction"]
    context = row["context"]
    response = row["response"]
    prompt = prompt_template.format(
        system_prompt=system_prompt, context_message=context, question_message = question
    )

    for temperature in np.arange(0.1, 2.0, 0.3):
        data = {
            "temperature": temperature,
            "model": model,
            "ground_truth_response": response,
            "prompt": str(prompt),
        }
        result_df = pd.concat([result_df, pd.DataFrame([data])], ignore_index=True)


result_df.to_excel(f"{module}_{quantization}_{test_num}.xlsx")

/tmp/ipykernel_1219024/1791041859.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([data])], ignore_index=True)


In [23]:
result_df = pd.read_excel(f"{module}_{quantization}_{test_num}.xlsx")
result_df["temperature"] = result_df["temperature"].round(2)

In [6]:
from utils.huggingfaceUtil import generate_response_hf

In [24]:
from utils.huggingfaceUtil import generate_response_hf
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import time  # Import the time module
 
csv_filename = f"{module}_{quantization}_{test_num}.csv"

pbar = tqdm(total=len(result_df))

responses = []

similarities = []
 
for index, row in result_df.iterrows():
    start_time = time.time()  # Record the start time
    response = generate_response_hf(row)
    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    updated_row = row.copy()
    response_str = response[0]["generated_text"]
    updated_row["full answer"] = response_str
    answer, label = extract_answer(response_str)
    updated_row["generated_response"] = answer
    updated_row["label"] = label
    updated_row["elapsed_time"] = elapsed_time  # Add elapsed time to the row
    # Add the timestamp to the updated_row
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    updated_row["timestamp"] = timestamp
    updated_dataframe = pd.DataFrame([updated_row])
    updated_dataframe.to_csv(csv_filename, index=False, mode="a", header=not index)
    pbar.update(1)
    
pbar.close()


  0%|          | 0/2100 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0.1


  0%|          | 1/2100 [00:05<2:56:56,  5.06s/it]

INFO:     127.0.0.1:45634 - "POST /generate-text HTTP/1.1" 200 OK
0.4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 2/2100 [00:10<2:57:09,  5.07s/it]

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  8%|▊         | 178/2100 [56:31<13:13:02, 24.76s/it]

INFO:     127.0.0.1:46094 - "POST /generate-text HTTP/1.1" 200 OK
1.0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  9%|▊         | 179/2100 [56:57<13:20:42, 25.01s/it]

INFO:     127.0.0.1:49512 - "POST /generate-text HTTP/1.1" 200 OK
1.3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
